<img src="https://www.pola.rs/share.jpg" width=400 height=200></img>

In [ ]:
# !pip install polars altair great-tables colorzero

In [ ]:
import polars as pl
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

# On the history of `DataFrame`

<img src="./df_hist.png" width=1000 height=500></img>

# Why the name "Polars"?

<font size=5>Because Pandas started it!</font>

<font size=5>Pandas: **Pan**el **Da**ta</font>

<font size=5>Polars: still an ursine, but with the `rs` suffix</font>

# Who?

https://www.ritchievink.com/

# Why?

### 1. Based on the Rust programming language, which is a very interesting language to know, might be very relevant for DS one day. polars has a python and a rust api.
### 2. FAST!
### 3. Can operate on datasets beyond the RAM size
### 4. Come for speed, stay for the API.

# A few words on Rust 

1. C/C++ performance 
2. Memory safety
3. Out of the box parallelism and concurrency
4. Data processing and ML: 
    - [Linfa](https://github.com/rust-ml/linfa) - scikit equivalent
    - [Burn](https://github.com/tracel-ai/burn) - DL package
    - [Polars](https://github.com/pola-rs/polars) - pandas equivalent
    
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/Rust_programming_language_black_logo.svg/2048px-Rust_programming_language_black_logo.svg.png" width=50 height=50></img>

<br>
<br>
<br>
<br>
 

## What we will cover:

- Benefits of polars, namely speed
- API differences from Pandas
- Eager API
- Lazy API
- SQL API (Yes, you can use polars directly with SQL)


# The main claim to fame: _speed_

[Link1](https://duckdblabs.github.io/db-benchmark/), [Link2](https://pola.rs/posts/benchmarks/)

## `polars` uses apache's Arrow internally which allows for optimized columnar storage

- `numpy`/`pandas` $\rightarrow$ `polars`: slow since a lot of conversion takes place...
- python primitives $\rightarrow$ `polars`: very fast
- binary files, parquet files... $\rightarrow$ `polars`: smart and fast

In [ ]:
%time _ = pl.Series([_ for _ in range(50_000_000)])

In [ ]:
# fails on 100M, while polars achieves it in ~6.2 seconds
%time _ = pd.Series([_ for _ in range(50_000_000)])

In [ ]:
arr = np.random.choice(['A', 'B', 'C'], size=100_000_000)

In [ ]:
%time pls = pl.Series(arr) # very slow due to conversion from numpy, better to work with primitives

In [ ]:
%time pds = pd.Series(arr)

In [ ]:
%time pls.max()

In [ ]:
%time pds.max()

## How long does it take to load a 5M row data set in Polars vs. Pandas? 

In [ ]:
_file_location = './limited-memory-example/yellow_tripdata_2015-01.csv'

In [ ]:
%time df_5m = pl.read_csv(_file_location, n_rows=5_000_000)

In [ ]:
%time pd_df = pd.read_csv(_file_location, nrows=5_000_000)

#### The full dataset contains 12,748,986 rows and 19 columns

#### Reading the full dataset with pandas will kill the kernel...

#### On polars, however...

In [ ]:
%time df = pl.read_csv(_file_location)

#### Forcing it to use the arrow parser is even shorter

In [ ]:
%time df = pl.read_csv(_file_location, use_pyarrow=True)

In [ ]:
type(df)

In [ ]:
df.shape

## polars DataFrames have `Schema`s

[`Schemas`](https://docs.pola.rs/api/python/dev/reference/dataframe/api/polars.DataFrame.schema.html) operate like `dict` with keys that are the column names and types that are part of `polars` magic.

Some key methods are:
- `names()` can also be accessed via `df.columns`
- `dtypes()`
- `values()`
- `items()`
- `keys()`
- `to_python()`- will return a `dict`

In [ ]:
df.schema

In [ ]:
df.columns # is equivalent to df.schema.names()

In [ ]:
df.estimated_size('gb')

## Dtypes

### `polars` uses columnar storage, since logically, every column has 1 type, having a declared type for each column allows for storing and treating each column separately and in an optimized way

In [ ]:
df.schema['VendorID']

In [ ]:
dt = df.schema['VendorID']

In [ ]:
type(type(dt))

In [ ]:
dt.min(), dt.max()

In [ ]:
dt.is_numeric()

In [ ]:
dt.is_decimal()

In [ ]:
dt.is_(df.schema['passenger_count'])

### dtypes can help us optimize

In [ ]:
df['store_and_fwd_flag'].unique()

In [ ]:
_c = df['store_and_fwd_flag'].cast(pl.Categorical()) # special data type for low cardinality

In [ ]:
%time df['store_and_fwd_flag'].value_counts()

In [ ]:
%time _c.value_counts()

# Eager Execution using `polars`

## Basic DataFrame methods

### Creating DataFrames ad hoc

In [ ]:
pl.DataFrame({'a': [1,2,3,4], 'b': [4,5,6,7]})

In [ ]:
# Our NYC dataset

df.head()

# Note that strings are quoted "some str"

## Some attributes

In [ ]:
df.height  # how many rows

In [ ]:
df.width  # how many columns

In [ ]:
df.flags  # holds the information if a column is sorted or not to avoid repeat sorting

### The `describe()` method

In [ ]:
%time df_5m.describe()

In [ ]:
%time pd_df.describe()

## Indexing: No `index` attribute, no `loc`/`iloc` methods... 
instead one canjust refer to row number (0 based indexing)

In [ ]:
df[0,0]

In [ ]:
df['VendorID']

In [ ]:
df[0,'VendorID']

In [ ]:
df[1000:1004,['tpep_dropoff_datetime', 'pickup_longitude']]

In [ ]:
type(df[0,:])

In [ ]:
type(df[:,0])

## A common `pandas` pattern is to use masks for filtering...

```python
df = pd.read_csv('/somewhere/something.csv')

mask = df['age'] > 30

df = df[mask] # would filter the DataFrame

```

## ... but it's not supported in `polars` ...

In [ ]:
mask = df['VendorID'] == 1

In [ ]:
df[mask, 'passenger_count'] # this should fail

## ... we'll get to `filter` later

## Other indexing methods `rows`/`iter_rows`/`rows_by_key`

In [ ]:
df_ex = pl.DataFrame(
     {
         "x": ["a", "b", "b", "a"],
         "y": [1, 2, 3, 4],
         "z": [0, 3, 6, 9],
     }
)

df_ex

### <font color='red'>Materialization Warning:</font> Use with caution: `rows`

Unlike `pandas`, `polars` does not store the values of the dataframe directly in memory, there is some disk usage that takes place and also the serialization is columnar, so when we call `rows()` it means that:<br>
1 - we materialize the entire dataframe<br>
2 - we circulate through each row and each column resulting in $O(N\times C)$ complexity<br>

In [ ]:
df_ex.rows() # a very expensive operation

In [ ]:
df_ex.rows(named=True) # or use df_ex.to_dicts()

### Memory effective option is using iterators: `iter_rows`

In [ ]:
for row in df_ex.iter_rows():
    print(row)

In [ ]:
_sum = 0
for row in df_ex.iter_rows():
    _sum += row[1]
    
print(f"the sum of the 2nd column is {_sum}")

In [ ]:
# value of last column in each row as a list
[row[-1] for row in df_ex.iter_rows()]

In [ ]:
# value of named in each row as a list
[row['x'] for row in df_ex.iter_rows(named=True)]

### <font color='red'>Materialization Warning:</font> `rows_by_key`

In [ ]:
df_ex

In [ ]:
df_ex.rows_by_key(key=['x'])

In [ ]:
df_ex.rows_by_key(key=['x'], named=True)

###  Unsure? Confused? You can always go back `to_pandas` or `to_numpy` 

In [ ]:
df_ex.to_pandas()

In [ ]:
df_ex.to_numpy()

## Expressions

### (The diagram is my own)

<img src="./api_layers.png" height=1200 width=400></img>

In [ ]:
c1 = pl.col("VendorID")

In [ ]:
c1

In [ ]:
type(c1)

In [ ]:
df.select(c1).head(3)

In [ ]:
df.select(pl.col("VendorID", "pickup_longitude"))

In [ ]:
df.select("VendorID", "pickup_longitude")

### Filtering using expressions

In [ ]:
mask = pl.col("pickup_longitude") > -73.

In [ ]:
mask # Not materialized, just the instruction

In [ ]:
type(mask)

In [ ]:
type((pl.col("pickup_longitude") > -73.) & (pl.col("passenger_count") < 1))

In [ ]:
df.filter(
    (pl.col("pickup_longitude") > -73.) &
    (pl.col("passenger_count") < 1)
)

### Adding columns - the wrong way (another common `pandas` pattern)

In [ ]:
df['x'] = 1 # this should fail

### Adding columns - the right way

In [ ]:
df = df.with_columns(pl.Series('x', [1]*df.shape[0]))

In [ ]:
df.select('x').head()

### Using literals: a safer way to work with constants as expressions

In [ ]:
_lit = pl.lit(1)
_lit

In [ ]:
type(_lit)

In [ ]:
df = df.with_columns(pl.lit(2).alias('y'))

In [ ]:
df.select('y', 'x', 'VendorID').head()

### Using expressions to add columns

In [ ]:
df_ex

In [ ]:
df_ex.select(pl.col('x'), pl.col('y')+1)

In [ ]:
df_ex.select( 
    
    (pl.col('y')/2).alias('y/2'),
    
    (2/pl.col('y')).alias('2/y'),
    
    (pl.col('y') + pl.col('z')).alias('y+z'),
    
    pl.col('z').arctan().alias('arctan(z)'),
    
    pl.col('z').arctan().ceil().alias('ceil(arctan(z))'),
    
    (pl.col('z')-5).abs().alias('abs(z-5)'),
    
    pl.col('z').is_in([1,2,3]).alias('z is in (1,2,3)'),
    
    pl.col('z').pow(2).alias('z**2'),
    
    (pl.col('y')**2).alias('y**2')
    
)

### Casting types

In [ ]:
df.select(pl.col('VendorID').cast(pl.String))

### But I really like my python function...

In [ ]:
def plus_2(x):
    return x+2

In [ ]:
df.select(
    original=pl.col('VendorID'), 
    plus_2=pl.col('VendorID').pipe(plus_2)
)

### Case when..

In [ ]:
# add a month_name column
df = df.with_columns(
    (pl.col("tpep_pickup_datetime").dt.strftime("%b").alias("month_name")),
    (pl.col("tpep_pickup_datetime").dt.strftime("%d").alias("month_day").cast(pl.Int64)),
)

In [ ]:
df.select('tpep_pickup_datetime', 'month_name', 'month_day')

In [ ]:
my_col = (
    pl.when(pl.col('month_day') <= 10)
      .then(pl.lit('first 10 days'))
      .when(pl.col('month_day') <= 20)
      .then(pl.lit('middle 10 days'))
      .otherwise(pl.lit('last 10 days'))
)

In [ ]:
type(my_col)

In [ ]:
df = df.with_columns(my_col.alias('day_group'))

In [ ]:
df.select('tpep_pickup_datetime', 'day_group')

### Removing columns

In [ ]:
df = df.drop('x') # or an iterator or a pointer to a pl.col

## Group by

<font color='red'>A note on api method names:</font> words will always be separated by an underscore `group_by`, `value_counts` etc...

### agg expressions

In [ ]:
pl.col('passenger_count').sum()

In [ ]:
type(pl.col('passenger_count').sum())

In [ ]:
df.group_by('month_name', 'day_group').agg(
    pl.len().alias('trips'), # count(1) like SQL
    pl.col('passenger_count').sum().alias('passengers'),
    pl.col('trip_distance').sum().alias('distance_driven'),
)

### `group_by_dynamic`

In [ ]:
result = (
    df.filter(pl.col('month_day') <= 3)
        .sort('tpep_pickup_datetime')
        .group_by_dynamic('tpep_pickup_datetime', every='1h')
        .agg(
            pl.len().alias('trips'),
            pl.col('passenger_count').sum().alias('passengers'),
            pl.col('trip_distance').sum().alias('distance_driven')
        )
)
result

In [ ]:
plt.figure(figsize=(8,3))
plt.plot(result['tpep_pickup_datetime'], result['trips'])
plt.grid()
plt.show()

<font color='red'>Note:</font>`polars` supports built-in plotting methods like `pandas`, but not using matplotlib. It uses `altair` instead, which works on `pandas` natively, not sure why this package was chosen for integration...

In [ ]:
result.plot.line(x='tpep_pickup_datetime', y='trips').properties(
    width=600,
    height=200
)

### SQL-like analytic functions are also available

In [ ]:
window = {
    'partition_by': [None],
    'order_by': ['tpep_pickup_datetime']
}

result = (
    df.filter(pl.col('month_day') == 1)
        .sort('tpep_pickup_datetime')
        .group_by_dynamic('tpep_pickup_datetime', every='1h')
        .agg(pl.len().alias('trips'))
        .with_columns(pl.col('trips').cum_sum()
                      .over(**window)
                      .alias('cumulative_trips'))
        .with_columns((pl.col('cumulative_trips') / pl.col('trips').sum().over([None])).alias('F'))
        .rename({'tpep_pickup_datetime':'hour'})
)
result

In [ ]:
_, ax = plt.subplots(figsize=(8,3))
ax.plot(result['hour'], result['F'])
ax.grid()

plt.show()

### Joins

In [ ]:
result1 = (
    df.filter(pl.col('month_day') == 1)
        .sort('tpep_pickup_datetime')
        .group_by_dynamic('tpep_pickup_datetime', every='1h')
        .agg(pl.len().alias('trips'))
        .with_columns(pl.col('tpep_pickup_datetime').dt.strftime('%H').alias('hour'))
)

result2 = (
    df.filter(pl.col('month_day') == 2)
        .sort('tpep_pickup_datetime')
        .group_by_dynamic('tpep_pickup_datetime', every='1h')
        .agg(pl.len().alias('trips'))
        .with_columns(pl.col('tpep_pickup_datetime').dt.strftime('%H').alias('hour'))
)

result1.join(result2, on='hour')

# Lazy API

So far we've been showing the 'eager' api, that performs calculations 'as they come'.

The **lazy** api works in a 'tell me what you want to do, and let me do it on my own' fashion

In [ ]:
lf = pl.scan_csv(_file_location)
lf

In [ ]:
type(lf)

In [ ]:
print(lf.explain())

### What the hell is $\pi */19$?

It's part of **relational algebra** (the math behind SQL), and it means:
- $\pi$ is the projection operator (in this case it just means 'select')
- ... the set of all rows * 
- ... over 19 columns

In [ ]:
lf = (
    pl.scan_csv(_file_location)
    .with_columns(pl.col('tpep_pickup_datetime').str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S"))
    .filter(pl.col('VendorID') == 1)
    .sort('tpep_pickup_datetime')
    .group_by_dynamic('tpep_pickup_datetime', every='1h')
    .agg(
        pl.len().alias('trips'),
        pl.col('passenger_count').sum().alias('passengers'),
        pl.col('trip_distance').sum().alias('distance_driven')
    )
)

In [ ]:
lf.show_graph()

The $\sigma$ operator is another of operation in Relational Algebra that is related to filters.

### What can we know about the LazyFrame without materializing it?

#### <font color='red'>Materialization Warning:</font> calling `columns` will perform materialization

In [ ]:
# should warn with PerformanceWarning
%time lf.columns

In [ ]:
# better to use 'collect_schema'
%time lf.collect_schema()

### But there's no data here, just instructions. So using indices for example, will result in an error

In [ ]:
lf[0:1,'trips'] # this should fail

### Materialization has to be explicit, using `collect` to get a `polars.DataFrame`

In [ ]:
%time lazy_result = lf.collect()

In [ ]:
def eager_time_test():
    return (
        pl.read_csv(_file_location)
        .with_columns(pl.col('tpep_pickup_datetime').str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S"))
        .filter(pl.col('VendorID') == 1)
        .sort('tpep_pickup_datetime')
        .group_by_dynamic('tpep_pickup_datetime', every='1h')
        .agg(
            pl.len().alias('trips'),
            pl.col('passenger_count').sum().alias('passengers'),
            pl.col('trip_distance').sum().alias('distance_driven')
        )
    )

In [ ]:
%time eager_result = eager_time_test()

In [ ]:
%time lazy_result = lf.collect() 

In [ ]:
lazy_result.head()

In [ ]:
eager_result.head()

In [ ]:
lazy_result.equals(eager_result)

### Profiling

In [ ]:
result, profile_result = lf.profile(show_plot=True)

In [ ]:
profile_result

In [ ]:
_, profile_result = lf.profile(show_plot=True, predicate_pushdown=False, projection_pushdown=False)

In [ ]:
profile_result

### Directly creating LazyDataFrame

In [ ]:
pl.DataFrame({'a': [1,2,3]}).lazy()

## SQL API (Lazy by definition)

<img src="./sql_api.png" width="400" height="400"></img>

In [ ]:
result1 = (
    df.filter(pl.col('month_day') == 1)
        .sort('tpep_pickup_datetime')
        .group_by_dynamic('tpep_pickup_datetime', every='1h')
        .agg(
            pl.len().alias('trips'),
            pl.col('passenger_count').sum().alias('passengers'),
            pl.col('trip_distance').sum().alias('distance_driven')
        )
    .with_columns(pl.col('tpep_pickup_datetime').dt.strftime('%H').alias('hour'))
)


In [ ]:
result1

In [ ]:
pl.sql("""select avg(distance_driven) as avg_distance from result1""")

In [ ]:
pl.sql("""select avg(distance_driven) as avg_distance from result1""").collect()

In [ ]:
result2 = (
    df.filter(pl.col('month_day') == 3) # <-- 2nd day
        .sort('tpep_pickup_datetime')
        .group_by_dynamic('tpep_pickup_datetime', every='1h')
        .agg(
            pl.len().alias('trips'),
            pl.col('passenger_count').sum().alias('passengers'),
            pl.col('trip_distance').sum().alias('distance_driven')
        )
    .with_columns(pl.col('tpep_pickup_datetime').dt.strftime('%H').alias('hour'))
)

In [ ]:
result = pl.sql("""
    select 
        t1.hour, 
        t1.distance_driven as dist1, 
        t2.distance_driven as dist2
    from result1 as t1 join result2 as t2 on t1.hour = t2.hour
    order by 1
""")

mat = result.collect()

In [ ]:
mat

In [ ]:
plt.figure(figsize=(8,3))
plt.plot(mat['hour'], mat['dist1'], label='day 1')
plt.plot(mat['hour'], mat['dist2'], label='day 2')
plt.legend()
plt.yscale('symlog')
plt.grid()
plt.show()

## If you don't want to run the entire computation, just to check if you did things correctly

### Use `head(n).collect()`

In [ ]:
result.head(2).collect()

## The SQL API also supports `pandas` DataFrames 

In [ ]:
pd_df = pd.DataFrame({'key':[1,2,3], 'a':[0,0,1]})
pl_df = pl.DataFrame({'key':[1,2,3], 'b':[1,0,0]})

pl.sql("""
    select
        pd_df.key,
        pd_df.a,
        pl_df.b,
        greatest(pd_df.a, pl_df.b) as max_ab,
        sum(greatest(pd_df.a, pl_df.b)) over(order by key) as cumsum_max_ab
    from
        pd_df join pl_df on pd_df.key = pl_df.key
""").collect()

## What is supported in SQL?

### The polars team claim they are trying to have everything that PostgreSQL support, but it's still wip

### What's supported right now

- `SELECT`
- `DISTINCT`
- `FROM`
- `JOIN (CROSS JOIN, FULL JOIN, INNER JOIN, [LEFT | RIGHT] [ANTI | SEMI] JOIN`, 
- `WHERE`
- `GROUP BY`
- `HAVING`
- `ORDER BY`
- `LIMIT/OFFSET`
- `EXCEPT`
- `INTERSECT`
- `UNION [ALL]`
- `UNNEST`
- `WITH`
- `OVER`
- Lots and lots of functions...

### Not supported

- `QUALIFY`

## Not covered: SQLContext, other connectors, batch read, caching

You can connect to CSVs, JSON files, Excel, AVRO, Feather, Parquet...

## Working with BigQuery

There is no direct connector to BigQuery, however, BigQuery queries can be transfered in Arrow format

```python
import polars as pl
from google.cloud import bigquery

client = bigquery.Client()

# Perform a query.
QUERY = """
    SELECT a,b,c
    FROM some_table 
    WHERE condition
    LIMIT 100""""

query_job = client.query(QUERY)  
rows = query_job.result() 

df = pl.from_arrow(rows.to_arrow())
```

# Summary:

1. Used correctly, `polars` is superior to `pandas` in terms of performance<br>
2. Not limited to RAM when using lazy execution<br>
2. SQL api, Lazy execution are very handy<br>
3. I personally prefer the functional expression API<br>
4. Multiple connectors are availble (parquet, Excel, CSV, ...)<br>
5. As of Dec 2024 numpy/pandas conversion happens behind the scenes when we fit models and plot stuff, so there is no speed up on scikit-learn/LightGBM/catboost/matplotlib - but, polars is moving quickly and we might see more and more direct integrations<br>
6. Rust - since this is A rapidly growing community, it might have some future for DS/Analytics too, and then polars will be rust's pandas equivalent<br>

## Bonus: `great-tables`

Integrates seamlessly with `polars` you can also use the `selectors` directly when selecting columns.

In [ ]:
from great_tables import GT, md, html, nanoplot_options
from colorzero import *

In [ ]:
_file_location = './limited-memory-example/yellow_tripdata_2015-01.csv'
df = pl.read_csv(_file_location)

In [ ]:
bw = (
    df
    .sort(pl.col('tpep_pickup_datetime'), descending=False)
    .group_by_dynamic(pl.col('tpep_pickup_datetime').str.to_datetime('%Y-%m-%d %H:%M:%S'), every="1w")
    .agg(
        pl.col('passenger_count').mean().alias('average_passenger_count'),
        pl.col('trip_distance').mean().alias('average_trip_distance'),
        pl.col('fare_amount').mean(),
        pl.col('extra').mean(),
        pl.col('mta_tax').mean(),
        pl.col('tip_amount').mean(),
        pl.col('tolls_amount').mean(),
        pl.col('total_amount').mean())
    ).rename(mapping={'tpep_pickup_datetime':'week'})

bw2 = (
    df
    .sort(pl.col('tpep_pickup_datetime'), descending=False)
    .group_by_dynamic(pl.col('tpep_pickup_datetime').str.to_datetime('%Y-%m-%d %H:%M:%S'), every="1d")
    .agg(pl.col('passenger_count').sum().alias('passenger_count_daily'))
    .rename(mapping={'tpep_pickup_datetime':'day'})
    .sort(pl.col('day'), descending=False)
    .group_by_dynamic(pl.col('day'), every="1w")
    .agg(pl.col('passenger_count_daily').implode())
    .with_columns(pl.col('passenger_count_daily').list.get(0))
    .rename(mapping={'day':'week'})
    )

gt1 = (
    bw.join(bw2, on='week', how='inner').style
        .tab_header('NYC Taxi dataset', subtitle=f"Weekly statistics between Dec 29, 2014 to Jan 26, 2015")
        .tab_stub(rowname_col='week')
        .tab_source_note(source_note='Kaggle, July 2017. https://www.kaggle.com/competitions/nyc-taxi-trip-duration')
        .tab_spanner(label='Averages', columns=['average_passenger_count', 'average_trip_distance'])
        .tab_spanner(label='Avg. Costs', columns=['fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount'])
        .fmt_currency(columns=['fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount'])
        .fmt_date(columns=['week'], date_style='m_day_year')
        .fmt_number(columns=['average_passenger_count', 'average_trip_distance'], compact=True)
        .fmt_nanoplot(columns='passenger_count_daily', autoscale=True)
        .cols_label(
            average_passenger_count=html('Passenger<br>count'),
            average_trip_distance=html('Trip<br>distance'),
            fare_amount=html('Fare'),
            extra='Extra',
            mta_tax='MTA tax',
            tip_amount='Tip',
            tolls_amount=html('Tolls'),
            total_amount=html('Total'),
            passenger_count_daily=html('Passenger count<br>(daily)')
        )
)
low_col = Color('lime') - Saturation(0.4)
high_col = Color('yellow') - Saturation(0.2)
gt2 = (
    gt1
    .data_color(
        palette=[low_col.html, high_col.html], 
        columns=[k for k,v in bw.schema.items() 
                 if v.is_numeric() 
                 and k not in ('average_passenger_count', 'average_trip_distance')])
    .data_color(
        palette=["white", "blue"], 
        columns=['average_passenger_count', 'average_trip_distance']
    )
)

In [ ]:
gt2